In [ ]:
from edgar import Company
company = Company("Oracle Corp", "0001341439")
tree = company.get_all_filings(filing_type = "10-K")
docs = Company.get_documents(tree, no_of_documents=5)

In [51]:
from edgar import Company, TXTML

company = Company("INTERNATIONAL BUSINESS MACHINES CORP", "0000051143")
docs = company.get_10Ks(5)

In [70]:
for doc in docs:
    print(doc.tag)

sec-document
sec-document
sec-document
sec-document
sec-document


In [71]:
for doc in docs[0].getchildren():
    print(doc.tag)

sec-header
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document
document

In [ ]:
docs[0].text_content()

In [ ]:
docs[0].getchildren()[1].getchildren()[0].text_content()

In [78]:
for doc in docs:
    print(doc.tag)
    for i in doc.getchildren()[1:]:
        print('\t', i.tag)
        for j in i.getchildren():
            print('\t\t', j.tag)
            print('\t\t', j.text)

sec-document
	 document
		 type
		 10-K

	 document
		 type
		 EX-3.2

	 document
		 type
		 EX-4.1

	 document
		 type
		 EX-10.1

	 document
		 type
		 EX-10.2

	 document
		 type
		 EX-13

	 document
		 type
		 EX-21

	 document
		 type
		 EX-23.1

	 document
		 type
		 EX-24.1

	 document
		 type
		 EX-24.2

	 document
		 type
		 EX-31.1

	 document
		 type
		 EX-31.2

	 document
		 type
		 EX-32.1

	 document
		 type
		 EX-32.2

	 document
		 type
		 GRAPHIC

	 document
		 type
		 GRAPHIC

	 document
		 type
		 GRAPHIC

	 document
		 type
		 GRAPHIC

	 document
		 type
		 GRAPHIC

	 document
		 type
		 EX-101.SCH

	 document
		 type
		 EX-101.CAL

	 document
		 type
		 EX-101.DEF

	 document
		 type
		 EX-101.LAB

	 document
		 type
		 EX-101.PRE

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		 XML

	 document
		 type
		